In [ ]:
import os
import cv2
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
figsize = (8, 6)
save_figures = False

In [ ]:
#path = '/scratch/ssd/cciw/Videos-20200114T160100Z-001/Quad1'
file = 'GLNI_12-1_2016-07-11_video-1.mp4'
#outpath = file.split('.')[0]

In [ ]:
#f = os.path.join(path, file)

In [ ]:
DATA_PATH = r'/scratch/gallowaa/cciw/Data'

# Search for all video files on Google Drive...
all_videos = glob(os.path.join(DATA_PATH, 'Videos_and_stills/GLNI/*/*/*/Videos/Quad*/*.mp4'))

videotable_path = os.path.join(DATA_PATH, 'Tables', 'QuadratVideos.csv')
video_df = pd.read_csv(videotable_path, index_col=0)

In [ ]:
vpath = video_df.iloc[video_df[video_df['Name'] == file].index]['Quadrat Video Path']

In [ ]:
tokens = video_df[video_df['Name'] == file]['Quadrat Video Path'].values[0].split('\\')
print(tokens)

In [ ]:
video_path = DATA_PATH + '/Videos_and_stills/GLNI'
for tok in tokens[4:-1]:
    video_path += '/' + tok

print(video_path)    
out_path = os.path.join(video_path, 'data')
video_path = os.path.join(video_path, file)

if not os.path.exists(out_path):
    os.mkdir(out_path)
    print('Making dir ', out_path)

In [ ]:
video_path

In [ ]:
'/scratch/gallowaa/cciw/Data/Videos_and_stills/GLNI/12/2016/Jul.11/Videos/Quad1'

In [ ]:
180 / np.pi * np.pi / 90

In [ ]:
#cap.read()

In [ ]:
FPS = 30
cap = cv2.VideoCapture(video_path) #cap.set(cv2.CAP_PROP_FPS, FPS)

In [ ]:
'''
for j in range(10):
    ret, im = cap.read()
currentFrame = j
'''
currentFrame = 0
for i in range(50):
    
    # Capture frame-by-frame
    ret, im = cap.read()
    
    if not ret: break

    # Saves image of the current frame in jpg file
    name = 'pframe_pi45_' + str(currentFrame) + '.jpg'
    save_path = os.path.join(out_path, name)
    print ('Creating...' + name)
    
    # Do processing
    #_, img = draw_lines(im, rho=1, theta=np.pi/2, mll=400, mlg=200, threshold=50, ds=1)    
    img = draw_lines(im, rho=1, theta=np.pi/45, mll=400, mlg=200, threshold=100, ds=1)
    cv2.imwrite(save_path, img)

    # To stop duplicate images
    currentFrame += 1

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
ret, im = cap.read()
plt.imshow(im)

In [ ]:
#from utils import draw_lines

In [ ]:
img = draw_lines(im, rho=1, theta=np.pi/90, mll=400, mlg=250, threshold=100, ds=1)

In [ ]:
#cv2.circle(img, (800, 400), 10, (255, 0, 0), thickness=2, lineType=8, shift=0)

plt.figure(figsize=figsize)
plt.imshow(img)
plt.tight_layout()
if save_figures:
    plt.savefig('img/' + outpath + '-Step-2.png')
plt.show()

In [ ]:
plt.imshow(edges)